# Hardware Testing Process — Rapport de laboratoire 

## Titre
Acquisition et analyse de signal

## Auteur
- Yildirim Arifcan

## Date
17-11-2025

## Objectifs
- Décrire le système expérimental et les instruments.
- Acquérir des signaux mesurables (tension, courant, capteur).
- Traiter et analyser les données (filtrage, transformée de Fourier, estimation de paramètres).
- Évaluer incertitudes et discuter les résultats.

## Matériel et instruments
- Oscilloscope: [DS1104]
- Générateur de fonctions: [DG1022]
- Câblage et adaptateurs

## Environnement
- Python


# Etape 1 : Détection automatique et gestion des instruments Rigol via PyVISA

Ce code a pour objectif d’automatiser la détection et l’utilisation des instruments RIGOL du laboratoire compatibles VISA, tels que le générateur de fonctions **DG1022** et l’oscilloscope **DS1104Z Plus**.
Il s’appuie sur trois éléments fondamentaux vus dans le syllabus :

* **Python** pour la programmation
* **VISA** (PyVISA + PyVISA-py) pour la communication matériel ↔ programme
* **SCPI** pour envoyer des commandes standardisées aux instruments

---

# 1) Classe `RigolInstrument`

Cette classe représente **un instrument Rigol individuel** (générateur, oscilloscope, etc.).

### ```python

class RigolInstrument:
def **init**(self, resource_name):
self.inst = pyvisa.ResourceManager('@py').open_resource(resource_name)
self.inst.timeout = 5000
self.inst.encoding = 'utf-8'
self.idn = self.get_idn()
self.instrument_type = self.classify_instrument()

````

### 🔍 Ce qui se passe ici :

1. **Ouverture de la ressource VISA**  
   `resource_name` ressemble à :  
   `USB0::0x1AB1::0x04CE::DS1ZC212301128::INSTR`  
   C’est l’identifiant unique du périphérique.

2. **Identification automatique de l’instrument**  
   Appel immédiat à :
   - `self.get_idn()` → récupère le `*IDN?`
   - `self.classify_instrument()` → détermine le type (oscillo/générateur)

---

# 2) Méthode `get_idn()`

```python
def get_idn(self):
    try:
        self.inst.write('*IDN?')
        time.sleep(0.1)
        return self.inst.read().strip()
    except pyvisa.errors.VisaIOError:
        return "Unknown"
````

### 🔍 Rôle :

* Envoie la commande SCPI standard `*IDN?`
* Lit la réponse, par ex. :
  `RIGOL TECHNOLOGIES,DS1104Z Plus,DS1ZC...,00.04.04.SP4`
* Si l’instrument ne répond pas → `"Unknown"`

Ce `*IDN?` est **le point de départ obligatoire** pour toute communication VISA (comme expliqué dans le syllabus).

---

# 3) Méthode `classify_instrument()`

```python
def classify_instrument(self):
    idn_upper = self.idn.upper()
    if "RIGOL" in idn_upper and "DG" in idn_upper:
        return "generator"
    elif "RIGOL" in idn_upper and ("DS" in idn_upper or "MSO" in idn_upper):
        return "oscilloscope"
    else:
        return "unknown"
```

### 🔍 Rôle :

Déterminer automatiquement le **type** d’instrument :

* `"DG"` → générateur DG1022
* `"DS"` ou `"MSO"` → oscilloscope (par ex. DS1104Z)

Cela permet de connecter automatiquement chaque appareil **sans hardcoder son adresse USB** → c’est exactement **l’exercice bonus** du syllabus (« Connect automatically one generator and one oscilloscope »).

---

# 4) Méthodes essentielles : `write`, `read`, `close`

```python
def write(self, command):
    self.inst.write(command)

def read(self):
    return self.inst.read()

def close(self):
    self.inst.close()
```

Ce sont des **wrappers** autour des fonctions PyVISA :

* `write(cmd)` : envoie une commande SCPI
* `read()` : lit une réponse
* `close()` : ferme proprement la connexion

Ces méthodes sont utilisées ensuite dans les fonctions du générateur et de l’oscilloscope.

---

# 5) Représentation textuelle

```python
def __str__(self):
    return f"{self.instrument_type.capitalize()} - IDN: {self.idn}"
```

Permet d’écrire :

```
Oscilloscope - IDN: RIGOL TECHNOLOGIES,DS1104Z Plus,...
```

---

# 6) Fonction `detect_rigol_instruments()`

```python
def detect_rigol_instruments():
    rm = pyvisa.ResourceManager('@py')
    devices = [d for d in rm.list_resources() if "USB" in d]

    generator = None
    oscilloscope = None
    print("Detecting Rigol instruments...")

    for dev in devices:
        rigol = RigolInstrument(dev)
        if rigol.instrument_type == "generator":
            generator = rigol
            print(f"Detected Generator: {generator}")
        elif rigol.instrument_type == "oscilloscope":
            oscilloscope = rigol
            print(f"Detected Oscilloscope: {oscilloscope}")

    return generator, oscilloscope
```

### 🔍 Rôle complet :

1. **Récupérer la liste des instruments VISA**
   Equivalent à :

   ```python
   rm.list_resources()
   ```

   Ce que le syllabus impose comme première étape.

2. **Filtrer les instruments USB**
   Les instruments du labo Rigol apparaissent au format `USB0::…`.

3. **Instancier la classe `RigolInstrument` pour chaque ressource**

4. **Classer automatiquement** :

   * Générateur DG1022
   * Oscilloscope DS1104Z
   * Autres

5. **Retourner un tuple** :

   ```python
   (generator, oscilloscope)
   ```

C’est exactement la solution demandée dans **le Challenge de la Session 1**.

---

# 🎯 Pourquoi ce code est utile pour le laboratoire ?

Il automatise plusieurs étapes :

### ✔ Identifier automatiquement les instruments

Plus besoin de copier manuellement l’adresse USB.

### ✔ Éviter les erreurs de connexion

Le code sait quel instrument est un oscilloscope et lequel est un générateur.

### ✔ Respecte complètement la logique VISA

* `ResourceManager`
* `open_resource()`
* `*IDN?`
* Chaînes SCPI

---

# Utilisation

> Nous avons développé une classe Python permettant de gérer automatiquement les instruments du laboratoire via PyVISA.
> Chaque instrument USB est détecté, interrogé via la commande SCPI `*IDN?`, puis classé selon son modèle (oscilloscope DS1104Z ou générateur DG1022).
> Cette structure permet d’abstraire la gestion matérielle et d’utiliser les instruments directement à travers des fonctions Python (write, read) tout en garantissant une communication fiable via le protocole VISA.

Exos 1 à 4 générateur :

In [ ]:

from tools.find_instruments import detect_rigol_instruments
import time

generator, oscilloscope = detect_rigol_instruments()

# ==================================== FONCTIONS POUR LE GÉNÉRATEUR DE SIGNAL ====================================


def generator_timed_signal():
    """Active et désactive un signal toutes les secondes pendant 10 secondes."""
    for i in range(10):
        generator.write("OUTP1 ON")
        time.sleep(1)
        generator.write("OUTP1 OFF")
        time.sleep(1)


def generator_sine_signal():
    """Configure une onde sinusoïdale sur Channel 1 et une onde carrée sur Channel 2."""

    # Sine wave sur Channel 1 avec amplitude 5 V, fréquence 3000 Hz et offset 0 V
    generator.write("APPL:SIN 3000,5,0")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    generator.write("OUTP1 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP1 OFF")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    
    # Square wave sur Channel 2 avec amplitude 3 V, fréquence 1000 Hz et offset 0 V
    generator.write(" APPLy:SQUare:CH2 1000,3,0")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    generator.write("OUTP:CH2 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP:CH2 OFF")



def generator_exponential_signal():
    """Configure une onde exponentielle sur Channel 1."""

    # Exponential wave sur Channel 1 avec amplitude 5 V
    generator.write("APPL:USER EXP_RISE,1,5,0")
    time.sleep(1)
    generator.write("OUTP1 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP1 OFF")

if __name__ == "__main__":
    if generator:
        print("Générateur de signal détecté.")
        #generator_timed_signal()
        #generator_sine_signal()
        #generator_exponential_signal()
        
    else:
        print("Générateur de signal non détecté !")

    generator.close()


Detecting Rigol instruments...
Générateur de signal non détecté !
Oscilloscope non détecté !


exo 5 :
To be complete, create a square signal on channel 1 that has
— a frequency of 1250 Hz
— an offset of 2.3V
— a duty-cycle of 60%
— an amplitude changing every 2 seconds form 2.3V to 4.6V

exo 6 : Record all the information of exercise 5 in a CSV file with a timeline and show it with a
graph

## Exos Oscilloscope :

1) Connect to the oscilloscope :
To start using the oscilloscope, let’s copy some functionalities and try to understand them by
executing
:

```
import pyvisa
rm = pyvisa.ResourceManager()
Osci = rm.open_resource('your ressource')
# To select a single channel (1 or 2) permanently you can use this line
# it allows you to not write CHAN2 at every call
Osci.write(':MEAS:SOUR:CHAN2')
data = []
data.append(Osci.query(':MEAS:Item? VPP'))
data.append(Osci.query(':MEAS:Item? VAVG'))
data.append(Osci.query(':MEAS:Item? PER'))
data.append(Osci.query(':MEAS:SOUR?'))
data.append(Osci.query(':CHAN1:SCAL?'))
data.append(Osci.query(':FUNC:WREC:FEND?'))
Osci.write(':MATH:FFT:SOUR CHAN1')
Osci.write(':MATH:OPER FFT')
Osci.write(':MATH:DISP ON')
data.append(Osci.query(':LAN:VISA?'))
# Warning to Measure 'MAth' should be off
# either it might not work, try it yourself! do you get data 8?
Osci.write(':MATH:OPER FFT')
Osci.write(':MATH:DISP OFF')
#measure the frequency of the signal
Osci.write(':MEAS:COUN:SOUR CHAN1')
data.append(Osci.query(':MEAS:COUN:VAL?'))
print(data)
```


2) Use Oscilloscope
    1. Create a program that can interface the oscilloscope and plot VRMS as a function of time.
    hints, to help you :
    (a) Use numpy
    (b) use Use MatplotLib 6

    2. To measure an accurate value of the VRMS take several values of it (10 then 100), plot all
    the value and the average of its. Don’t forget to leave some time to the data to be stored
    (500ms per scan).

    3. Let’s start reducing the value of waiting time. Compute the minimum waiting time and
    with it the sampling rate of VRMS on the oscilloscope.

3. Full wave measurement

Try this code that allows you to read what you see on the oscilloscope. How does it perform ?

In [6]:
import pyvisa
import numpy as n
import matplotlib.pyplot as mp
#Create a resource manager
8
resources = pyvisa.ResourceManager()
#Open the Rigol by name. (Change this to the string for your instrument)
osc = resources.open_resource('USB0::0x1AB1::0x04CE::DS1ZC212301108::INSTR')
#Return the Rigol's ID string to tell us it's there
print(osc.query('*IDN?'))
# reading procedure initialisation
osc.write(':WAVeform:FORMat ASCII') #easy to read information but slower
osc.write(':WAVeform:Mode NORMAL') #see programming book
L1 = osc.query(':WAVeform:PREamble?') #to know everything
print(L1)
osc.write(':WAVform:SOURce Channel1')
D1 = osc.query(':WAVEFORM:DATA?')
D2 = D1.split(',')
Lgth = n.size(D2)
mp.plot(n.linspace(0,1,Lgth-1),D2[1:Lgth])

ValueError: No device found.

What code could help you get a better plot of your data ? Print D1 or D2, this might help you to
find the answer. Hint : the ascii format is not convenient to plot your data , try to cast it into an
usable one

4. phase measurement
Based on two identical sinusoidal signals but having a difference in phase, perform a delay and
phase measurement. Does it correspond to the signal generated by the rigol generator ? To go
further, you can generate the signal in Python with the DG1022.

5. Automated code
Create a generic code that generates a sinus with the DG1022 and measure it with the DS1104.
The measured value must be plotted with Matplotlib, in an understandable voltage and time scale.
The legend of the plot should have the frequency, VPP, Vmin and Vmax. At the same time, the
values generated and measured have to be stored in a TXT or CSV file for later use.